# Create a search index in Azure AI Search using the Azure SDK for Python

This notebook steps through creating, loading, and querying an index in Azure AI Search index by calling the azure-search-documents library in the Azure SDK for Python. 

## Install packages and set variables

In [1]:
! pip install azure-search-documents==11.6.0b12 --quiet
! pip install azure-identity --quiet
! pip install python-dotenv --quiet

In [ ]:
# Provide variables
search_endpoint: str = "https://quickstartsearchservice.search.windows.net"
search_api_key: str = "YOUR_SEARCH_API_KEY"
index_name: str = "hotels-sample-index"

## Create an index

In [5]:
from azure.core.credentials import AzureKeyCredential

# Create a credential object
credential = AzureKeyCredential(search_api_key)

# Create a search schema
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchableField,
    ComplexField,
    SearchFieldDataType,
)

# Create a SearchIndexClient
index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)

fields = [
        SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),

        SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),

        SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=True),
        SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),
        SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),

        ComplexField(name="Address", fields=[
            SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
            SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        ])
    ]

# Define empty scoring profiles and suggesters
scoring_profiles = []
suggesters = [{"name": "sg", 
               "source_fields": ["Tags", "Address/City", "Address/Country"]}]

# Create the search index
index = SearchIndex(name=index_name, fields=fields, scoring_profiles=scoring_profiles, suggesters=suggesters)
result = index_client.create_index(index)

print(f"Index '{result.name}' created successfully.")

Index 'hotels-quickstart-csharp' created successfully.


## Create a documents payload

In [6]:
# Create a documents payload
documents = [
    {
    "@search.action": "upload",
    "HotelId": "1",
    "HotelName": "Stay-Kay City Hotel",
    "Description": "This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.",
    "Category": "Boutique",
    "Tags": [ "view", "air conditioning", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "2020-01-18T00:00:00Z",
    "Rating": 3.60,
    "Address": {
        "StreetAddress": "677 5th Ave",
        "City": "New York",
        "StateProvince": "NY",
        "PostalCode": "10022",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "2",
    "HotelName": "Old Century Hotel",
    "Description": "The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, beer dinners, and live music.",
    "Category": "Boutique",
    "Tags": [ "pool", "free wifi", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "2019-02-18T00:00:00Z",
    "Rating": 3.60,
    "Address": {
        "StreetAddress": "140 University Town Center Dr",
        "City": "Sarasota",
        "StateProvince": "FL",
        "PostalCode": "34243",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "3",
    "HotelName": "Gastronomic Landscape Hotel",
    "Description": "The Gastronomic Hotel stands out for its culinary excellence under the management of William Dough, who advises on and oversees all of the Hotel’s restaurant services.",
    "Category": "Suite",
    "Tags": [ "restaurant", "bar", "continental breakfast" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "2015-09-20T00:00:00Z",
    "Rating": 4.80,
    "Address": {
        "StreetAddress": "3393 Peachtree Rd",
        "City": "Atlanta",
        "StateProvince": "GA",
        "PostalCode": "30326",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "4",
    "HotelName": "Sublime Palace Hotel",
    "Description": "Sublime Palace Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 19th century resort, updated for every modern convenience.",
    "Category": "Boutique",
    "Tags": [ "concierge", "view", "air conditioning" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "2020-02-06T00:00:00Z",
    "Rating": 4.60,
    "Address": {
        "StreetAddress": "7400 San Pedro Ave",
        "City": "San Antonio",
        "StateProvince": "TX",
        "PostalCode": "78216",
        "Country": "USA"
        }
    }
]

## Upload documents

In [7]:
# Create a SearchClient 
search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=credential)

# Upload documents to the index
try:
    result = search_client.upload_documents(documents=documents)
    print(f"Uploaded {len(result)} documents to index '{index_name}'.")

except Exception as e:
    print(f"Error uploading documents: {e.message}")

Uploaded 4 documents to index 'hotels-quickstart-csharp'.


## Run your first query

In [ ]:
# Run an empty query (returns selected fields, all documents)
result = search_client.search(query_type="simple",
                              search_text="*",
                              select="HotelName, Description, Tags",
                              include_total_count=True)

# Display the total count of documents found
print("Total documents found: ", result.get_count())
for doc in result: 
    print(f"search score: {doc['@search.score']}")
    print(f"HotelName: {doc['HotelName']}\n - Tags: {doc['Tags']}\n - Description: {doc['Description']}")


Total documents found:  4
search score: 1.0
HotelName: Gastronomic Landscape Hotel
 - Tags: ['restaurant', 'bar', 'continental breakfast']
 - Description: The Gastronomic Hotel stands out for its culinary excellence under the management of William Dough, who advises on and oversees all of the Hotel’s restaurant services.
search score: 1.0
HotelName: Old Century Hotel
 - Tags: ['pool', 'free wifi', 'concierge']
 - Description: The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, beer dinners, and live music.
search score: 1.0
HotelName: Sublime Palace Hotel
 - Tags: ['concierge', 'view', 'air conditioning']
 - Description: Sublime Palace Hotel is located in the heart of the historic center of Sublime in an extremely vibr

## Run a term query

In [14]:
# Run a term query
result = search_client.search(query_type="simple",
                              search_text="wifi",
                              select="HotelName, Description, Tags",
                              include_total_count=True)

# Display the total count of documents found
print("Total documents found: ", result.get_count())
for doc in result: 
    print(f"search score: {doc['@search.score']}")
    print(f"HotelName: {doc['HotelName']}\n - Tags: {doc['Tags']}\n - Description: {doc['Description']}")


Total documents found:  1
search score: 0.6931472
HotelName: Old Century Hotel
 - Tags: ['pool', 'free wifi', 'concierge']
 - Description: The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, beer dinners, and live music.


## Add a filter

In [ ]:
# Add a filter
result = search_client.search(
                              search_text="hotels",
                              select="HotelId, HotelName, Description, Rating",
                              filter="Rating ge 3",
                              order_by="Rating desc",
                              include_total_count=True)

# Display the total count of documents found
for doc in result:
    print("{} : {}-{} rating".format(doc['HotelId'], doc['HotelName'], doc['Rating']))

3 : Gastronomic Landscape Hotel-4.8 rating
4 : Sublime Palace Hotel-4.6 rating
2 : Old Century Hotel-3.6 rating
1 : Stay-Kay City Hotel-3.6 rating


## Scope a query to specific searchable fields

In [17]:
results = search_client.search(
                                search_text="sublime",
                                search_fields=["HotelName" ],
                                select="HotelId, HotelName",
                                include_total_count=True
                            )

for doc in results:
    print("{} : {}".format(doc['HotelId'], doc['HotelName']))


4 : Sublime Palace Hotel


## Return facets

In [20]:
# Return facets
results = search_client.search( search_text="*",
                                facets=["Category"],
                                include_total_count=True
                                )

facets = results.get_facets()
for facet in facets["Category"]:
    print(f"{facet}")
    

{'value': 'Boutique', 'count': 3}
{'value': 'Suite', 'count': 1}


## Look up a document 

In [21]:
# Look up a specific document by ID
results = search_client.get_document(key="3")

print("Details for hotel ID 3:")
for key, value in results.items():
    print(f"  {key}: {value}")

Details for hotel ID 3:
  HotelId: 3
  HotelName: Gastronomic Landscape Hotel
  Description: The Gastronomic Hotel stands out for its culinary excellence under the management of William Dough, who advises on and oversees all of the Hotel’s restaurant services.
  Category: Suite
  Tags: ['restaurant', 'bar', 'continental breakfast']
  ParkingIncluded: True
  LastRenovationDate: 2015-09-20T00:00:00Z
  Rating: 4.8
  Address: {'StreetAddress': '3393 Peachtree Rd', 'City': 'Atlanta', 'StateProvince': 'GA', 'PostalCode': '30326', 'Country': 'USA'}


## Autocomplete a query

In [25]:
# Autocomplete a query
search_suggestions = "New y"
results = search_client.autocomplete(search_text=search_suggestions,
                                    suggester_name="sg",
                                    mode="twoTerms")
print("Autocomplete suggestions for '{}':".format(search_suggestions))
for suggestion in results:
    print(" - {}".format(suggestion['text']))

Autocomplete suggestions for 'New y':
 - york


## Clean up

If you are finished with this index, you can delete it by running the following lines. Deleting unnecessary indexes frees up space for stepping through more quickstarts and tutorials.

In [26]:
# Delete the index
try:
    result = index_client.delete_index(index_name)
    print(f"Index '{index_name}' deleted successfully.")
except Exception as e:
    print(f"Error deleting index '{index_name}': {e.message}")


Index 'hotels-quickstart-csharp' deleted successfully.


Confirm the index deletion by running the following script that lists all of the indexes on your search service. If hotels-quickstart is not listed, you've successfully deleted the index and have completed this quickstart.

In [27]:
try: 
    result = index_client.get_index(index_name)
    print(f"Index '{index_name}' exists.")
except Exception as e:
    print(f"Index '{index_name}' does not exist: {e.message}")

Index 'hotels-quickstart-csharp' does not exist: () No index with the name 'hotels-quickstart-csharp' was found in the service 'aisearch-ktds712'.
Code: 
Message: No index with the name 'hotels-quickstart-csharp' was found in the service 'aisearch-ktds712'.
